In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [18]:
class GoogleNet(nn.Module):
    def __init__(self, in_channels=3, num_classes=1000):
        super(GoogleNet, self).__init__()
        self.conv1=Conv_Block(in_channels=in_channels, out_channels=64, kernel_size=(7,7),stride=(2,2), padding=(3,3))
        self.maxpool1=nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.conv2=Conv_Block(64,192, kernel_size=(3,3), stride=1, padding=1)
        
        self.maxpool2=nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.inception3a=Inception_Block(192,64,96,128,16,32,32)
        self.inception3b=Inception_Block(256,128,128,192,32,96,64)
        
        self.maxpool3=nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.inception4a=Inception_Block(480, 192, 96, 208, 16, 48, 64)
        self.inception4b=Inception_Block(512, 160, 112, 224, 24, 64, 64)
        self.inception4c=Inception_Block(512, 128, 128, 256, 24, 64, 64)
        self.inception4d=Inception_Block(512, 112, 144, 288, 32, 64, 64)
        self.inception4e=Inception_Block(528, 256, 160, 320, 32, 128, 128)
                                         
        self.maxpool4=nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.inception5a=Inception_Block(832, 256, 160, 320, 32, 128, 128)
        self.inception5a=Inception_Block(832, 384, 192, 384, 48, 128, 128)
        
        self.avgpool=nn.AvgPool2d(kernel_size=7, stride=1)
        self.dropout=nn.Dropout(p=0.4)
        self.fc1=nn.Linear(1024,1000)
    def forward(self,x):
        x=self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        # x = self.conv3(x)
        x = self.maxpool2(x)

        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)

        x = self.inception4a(x)

        # Auxiliary Softmax classifier 1
        #if self.aux_logits and self.training:
            #aux1 = self.aux1(x)

        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)

        # Auxiliary Softmax classifier 2
        #if self.aux_logits and self.training:
            #aux2 = self.aux2(x)

        x = self.inception4e(x)
        x = self.maxpool4(x)
        x = self.inception5a(x)
        x = self.inception5b(x)
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.dropout(x)
        x = self.fc1(x)
        return x

        #if self.aux_logits and self.training:
            #return aux1, aux2, x
        #else:
            #return x                                
                                         
                                         
        

In [19]:
class Inception_Block(nn.Module):
    def __init__(self, in_channels, out_1x1, reduction_3x3, out_3x3, reduction_5x5, out_5x5, out_1x1pool):
        super(Inception_Block, self).__init__()
        
        self.branch1=Conv_Block(in_channels, out_1x1, kernel_size=1)
        self.branch2=nn.Sequential(Conv_Block(in_channels, reduction_3x3, kernel_size=1),
                                  Conv_Block(reduction_3x3,out_3x3, kernel_size=3,stride=1, padding=1)
                                  )
        self.branch3=nn.Sequential(Conv_Block(in_channels,reduction_5x5, kernel_size=1),
                                   Conv_Block(reduction_5x5, out_5x5, kernel_size=5, stride=1, padding=1)
                                  )
        self.branch4=nn.Sequential(nn.MaxPool2d(kernel_size=3, stride=1,padding=1),
                                   Conv_Block(in_channels, out_1x1pool, kernel_size=1))
    def forword(self,x):
        return torch.cat([self.branch1(x), self.branch2(x),self.branch3(x),self.branch4(x)],1)
        
        
        
        
        

In [20]:
class Conv_Block(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):#
        super(Conv_Block, self).__init__()
        self.relu=nn.ReLU()
        self.conv=nn.Conv2d(in_channels, out_channels, **kwargs)# kernel_size=(1,1),(3,3),(5,5)
        self.batchnorm=nn.BatchNorm2d(out_channels)
    def forward(self, x):
        return self.relu(self.batchnorm(self.conv))
        

In [24]:
if __name__=="__main__":
    x=torch.randn(3,3,224,224)
    model=GoogleNet()
    print(model)
    #print(model(x).shape)
    

GoogleNet(
  (conv1): Conv_Block(
    (relu): ReLU()
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv2): Conv_Block(
    (relu): ReLU()
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batchnorm): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (inception3a): Inception_Block(
    (branch1): Conv_Block(
      (relu): ReLU()
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
      (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (branch2): Sequential(
      (0): Conv_Block(
        (relu): ReLU()
        (conv): Conv2d(192, 96, kernel_size